In [3]:
pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   --------- ------------------------------ 61.4/250.0 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import re

def clean_name(name):
    """
    Clean the name by removing unwanted characters and dates.
    """
    name = re.sub('[^\x00-\x7F]+', '', name)  # Remove non-ASCII characters
    name = re.sub('\(\d{4}-\d{4}\)|\(\d{4}\)', '', name)  # Remove dates
    return name.strip()

def process_entry_v5(entry):
    """
    Process each entry with enhanced rules for corporate identification and name cleaning.
    Converts any non-string input into a string to handle NaN values.
    """
    # Convert entry to string to handle NaN values gracefully
    entry = str(entry)
    
    if 'unidentified' in entry.lower():
        return [("<flag>", "<flag>", "<flag>")]
    
    #corporate_keywords = [
    #    'Group', 'Company', 'Corporation', 'Incorporated', 'Inc.', 'LLC', 'Ltd.',
    #    'and', '&', 'Architects', 'Architecture', 'Studio', 'Associates', 'Bureau',
    #    'Office of Supervising Architect', 'Engineering', 'Construction', 'Consultants',
    #    'Design', 'Partnership', 'Firm', 'PLLC', 'Office of',
    #    'Skidmore Owings Merrill', 'Venturi Rauch & Scott Brown',
    #]
    corporate_keywords = [
    'Group', 'Company', 'Corporation', 'Incorporated', 'Inc.', 'LLC', 'Ltd.',
    'and', '&', 'Architects', 'Architecture', 'Studio', 'Associates', 'Bureau',
    'Office of Supervising Architect', 'Engineering', 'Construction', 'Consultants',
    'Design', 'Partnership', 'Firm', 'PLLC', 'Office of',
    'Skidmore Owings Merrill', 'Venturi Rauch & Scott Brown', 'architectural firm',
    'architects', 'architecture', 'engineering', 'construction', 'design studio'
    ]

    
    entry = clean_name(entry)
    primary_entities = [e.strip() for e in entry.split(';')]
    processed_entries = []
    
    for entity in primary_entities:
        if '|' in entity:
            name, role = [part.strip() for part in entity.split('|', 1)]
        else:
            name, role = entity, 'contributor'
        
        entity_type = 'corporate' if any(keyword.lower() in name.lower() for keyword in corporate_keywords) else 'person'
        processed_entries.append((name, role, entity_type))
    
    return processed_entries

def expand_entries(row):
    """
    Expand processed entries into separate columns within the same row.
    """
    expanded_data = {}
    for i, (name, role, entity_type) in enumerate(row['Processed'], start=1):
        expanded_data[f'Primary archt/firm[{i}][Name]'] = name
        expanded_data[f'Primary archt/firm[{i}][Role]'] = role
        expanded_data[f'Primary archt/firm[{i}][Type]'] = entity_type
    return pd.Series(expanded_data)

def process_and_save_excel(input_excel_path, output_excel_path, unique_output_path):
    """
    Load the input Excel, process it with the latest rules, save the outputs,
    and also save a second output with unique values for specified columns.
    """
    df = pd.read_excel(input_excel_path)
    
    # Apply the 'process_entry_v5' function to the 'Primary archt/firm' column
    df['Processed'] = df['Primary archt/firm'].apply(process_entry_v5)
    
    # Expand processed entries
    expanded_df = df.apply(expand_entries, axis=1)
    
    # Concatenate the original dataframe with the expanded dataframe
    result_df = pd.concat([df, expanded_df], axis=1).drop(columns=['Processed'])
    
    # Save the result to an Excel file
    result_df.to_excel(output_excel_path, index=False)
    print(f"All data saved to {output_excel_path}")

    # Select columns for the unique data output
    columns_to_keep = [ 'Primary archt/firm'] + \
                      [col for col in result_df.columns if col.startswith('Primary archt/firm[')]
    unique_df = result_df[columns_to_keep].drop_duplicates()
    
    # Save the unique entries to another Excel file
    unique_df.to_excel(unique_output_path, index=False)
    print(f"Unique data saved to {unique_output_path}")

# Example usage
input_excel_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\AAA_Proj_db_20231019-MSITM_Architech_curated.xlsx'
output_excel_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\AAA_Proj_db_20231019-MSITM_Architech_curated_output.xlsx'
unique_output_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\Unique_AAA_Proj_db_20231019-MSITM_Architech.xlsx'
process_and_save_excel(input_excel_path, output_excel_path, unique_output_path)



All data saved to C:\Users\angel\OneDrive\Desktop\Capstone\AAA_Proj_db_20231019-MSITM_Architech_curated_output.xlsx
Unique data saved to C:\Users\angel\OneDrive\Desktop\Capstone\Unique_AAA_Proj_db_20231019-MSITM_Architech.xlsx


In [ ]:
import pandas as pd
import re

def clean_name(name):
    """
    Clean the name by removing unwanted characters and dates.
    """
    name = re.sub('[^\x00-\x7F]+', '', name)  # Remove non-ASCII characters
    name = re.sub('\(\d{4}-\d{4}\)|\(\d{4}\)', '', name)  # Remove dates
    return name.strip()

def process_entry_v5(entry):
    """
    Process each entry with enhanced rules for corporate identification and name cleaning.
    Converts any non-string input into a string to handle NaN values.
    """
    entry = str(entry)
    
    if 'unidentified' in entry.lower():
        return [("", "", "")]
    
    corporate_keywords = [
        'Group', 'Company', 'Corporation', 'Incorporated', 'Inc.', 'LLC', 'Ltd.',
        'and', '&', 'Architects', 'Architecture', 'Studio', 'Associates', 'Bureau',
        'Office of Supervising Architect', 'Engineering', 'Construction', 'Consultants',
        'Design', 'Partnership', 'Firm', 'PLLC', 'Office of',
        'Skidmore Owings Merrill', 'Venturi Rauch & Scott Brown', 'architectural firm',
        'architects', 'architecture', 'engineering', 'construction', 'design studio'
    ]
    
    entry = clean_name(entry)
    primary_entities = [e.strip() for e in entry.split(';')]
    processed_entries = []
    
    for entity in primary_entities:
        if '|' in entity:
            name, role = [part.strip() for part in entity.split('|', 1)]
        else:
            name, role = entity, 'contributor'
        
        entity_type = 'corporate' if any(keyword.lower() in name.lower() for keyword in corporate_keywords) else 'person'
        processed_entries.append((name, role, entity_type))
    
    return processed_entries

def expand_entries(row):
    """
    Expand processed entries into separate columns within the same row.
    """
    expanded_data = {}
    for i, (name, role, entity_type) in enumerate(row['Processed'], start=1):
        expanded_data[f'Primary archt/firm[{i}][Name]'] = name
        expanded_data[f'Primary archt/firm[{i}][Role]'] = role
        expanded_data[f'Primary archt/firm[{i}][Type]'] = entity_type
    return pd.Series(expanded_data)

def process_and_save_csv(input_csv_path, output_csv_path):
    """
    Load the input CSV, process it with the latest rules, and save the outputs.
    """
    df = pd.read_csv(input_csv_path)
    
    df['Processed'] = df['Primary archt/firm'].apply(process_entry_v5)
    expanded_df = df.apply(expand_entries, axis=1)
    result_df = pd.concat([df, expanded_df], axis=1).drop(columns=['Processed'])
    
    result_df.to_csv(output_csv_path, index=False)
    print(f"All data saved to {output_csv_path}")

# Example usage
input_csv_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\AAA_Proj_db_20231019-MSITM_Architech_curated.csv'
output_csv_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\AAA_Proj_db_20231019-MSITM_Architech_curated_output.csv'
process_and_save_csv(input_csv_path, output_csv_path)


In [6]:
import pandas as pd
import re

def clean_name(name):
    """
    Clean the name by removing unwanted characters and dates.
    """
    name = re.sub('[^\x00-\x7F]+', '', name)  # Remove non-ASCII characters
    name = re.sub('\(\d{4}-\d{4}\)|\(\d{4}\)', '', name)  # Remove dates
    return name.strip()

def process_entry_v5(entry):
    """
    Process each entry with enhanced rules for corporate identification and name cleaning.
    Converts any non-string input into a string to handle NaN values.
    Returns both processed entries and a flag indicating processing status.
    """
    entry = str(entry)
    flag = 0  # Default flag, 0 means no issues

    if 'unidentified' in entry.lower():
        return [("", "", "")], 0  # Set flag to 1 if problematic

    corporate_keywords = [
        'Group', 'Company', 'Corporation', 'Incorporated', 'Inc.', 'LLC', 'Ltd.',
        'and', '&', 'Architects', 'Architecture', 'Studio', 'Associates', 'Bureau',
        'Office of Supervising Architect', 'Engineering', 'Construction', 'Consultants',
        'Design', 'Partnership', 'Firm', 'PLLC', 'Office of',
        'Skidmore Owings Merrill', 'Venturi Rauch & Scott Brown', 'architectural firm',
        'architects', 'architecture', 'engineering', 'construction', 'design studio'
    ]

    entry = clean_name(entry)
    primary_entities = [e.strip() for e in entry.split(';')]
    processed_entries = []

    for entity in primary_entities:
        if '|' in entity:
            name, role = [part.strip() for part in entity.split('|', 1)]
        else:
            name, role = entity, 'contributor'
        
        entity_type = 'corporate' if any(keyword.lower() in name.lower() for keyword in corporate_keywords) else 'person'
        processed_entries.append((name, role, entity_type))
    
    return processed_entries, flag

def expand_entries(row):
    """
    Expand processed entries into separate columns within the same row.
    Includes processing flag.
    """
    expanded_data = {}
    entries, flag = row['Processed']
    for i, (name, role, entity_type) in enumerate(entries, start=1):
        expanded_data[f'Primary archt/firm[{i}][Name]'] = name
        expanded_data[f'Primary archt/firm[{i}][Role]'] = role
        expanded_data[f'Primary archt/firm[{i}][Type]'] = entity_type
    expanded_data['Primary archt/firm_flag'] = flag
    return pd.Series(expanded_data)

def process_and_save_csv(input_csv_path, output_csv_path):
    """
    Load the input CSV, process it with the latest rules, and save the outputs.
    """
    df = pd.read_csv(input_csv_path)
    
    df['Processed'] = df['Primary archt/firm'].apply(process_entry_v5)
    expanded_df = df.apply(expand_entries, axis=1)
    result_df = pd.concat([df, expanded_df], axis=1).drop(columns=['Processed'])
    
    result_df.to_csv(output_csv_path, index=False)
    print(f"All data saved to {output_csv_path}")

# Example usage
input_csv_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\Primary_archtfirm\\AAA_Proj_db_20231019-MSITM_Architech_input.csv'
output_csv_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\Primary_archtfirm\\AAA_Proj_db_20231019-MSITM_Architech_curated_output.csv'
process_and_save_csv(input_csv_path, output_csv_path)


C:\Users\angel\AppData\Local\Temp\ipykernel_41236\2730285286.py:66: DtypeWarning: Columns (35,43,47,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv_path)


All data saved to C:\Users\angel\OneDrive\Desktop\Capstone\Primary_archtfirm\AAA_Proj_db_20231019-MSITM_Architech_curated_output.csv


In [11]:
import pandas as pd
import re

def clean_name(name):
    """
    Clean the name by removing unwanted characters and dates.
    """
    name = re.sub('[^\x00-\x7F]+', '', name)  # Remove non-ASCII characters
    name = re.sub('\(\d{4}-\d{4}\)|\(\d{4}\)', '', name)  # Remove dates
    return name.strip()

def process_entry_v5(entry):
    """
    Process each entry with enhanced rules for corporate identification and name cleaning.
    Converts any non-string input into a string to handle NaN values.
    Returns both processed entries and a flag indicating processing status.
    """
    entry = str(entry)
    flag = 0  # Default flag, 0 means no issues

    if 'unidentified' in entry.lower():
        return [("", "", "")], 0  # Return empty values with flag set if problematic

    corporate_keywords = [
        'Group', 'Company', 'Corporation', 'Incorporated', 'Inc.', 'LLC', 'Ltd.',
        'and', '&', 'Architects', 'Architecture', 'Studio', 'Associates', 'Bureau',
        'Office of Supervising Architect', 'Engineering', 'Construction', 'Consultants',
        'Design', 'Partnership', 'Firm', 'PLLC', 'Office of',
        'Skidmore Owings Merrill', 'Venturi Rauch & Scott Brown', 'architectural firm',
        'architects', 'architecture', 'engineering', 'construction', 'design studio'
    ]

    entry = clean_name(entry)
    primary_entities = [e.strip() for e in entry.split(';')]
    processed_entries = []

    for entity in primary_entities:
        if '|' in entity:
            name, role = [part.strip() for part in entity.split('|', 1)]
        else:
            name, role = entity, 'contributor'
        
        entity_type = 'corporate' if any(keyword.lower() in name.lower() for keyword in corporate_keywords) else 'person'
        processed_entries.append((name, role, entity_type))
    
    return processed_entries, flag

def expand_entries(row):
    """
    Expand processed entries into separate columns within the same row.
    Includes processing flag.
    """
    expanded_data = {}
    entries, flag = row['Processed']
    for i, (name, role, entity_type) in enumerate(entries, start=1):
        expanded_data[f'Primary archt/firm[{i}][Name]'] = name
        expanded_data[f'Primary archt/firm[{i}][Role]'] = role
        expanded_data[f'Primary archt/firm[{i}][Type]'] = entity_type
    expanded_data['Primary archt/firm_flag'] = flag
    return pd.Series(expanded_data)

def process_and_save_csv(input_csv_path, output_csv_path):
    """
    Load the input CSV, process it with the latest rules, and save the outputs.
    """
    df = pd.read_csv(input_csv_path)
    
    df['Processed'] = df['Primary archt/firm'].apply(process_entry_v5)
    expanded_df = df.apply(expand_entries, axis=1)
    
    # Concatenate the original dataframe with the expanded dataframe
    result_df = pd.concat([df, expanded_df], axis=1)
    
    # Drop the 'Processed' column which is no longer needed
    result_df = result_df.drop(columns=['Processed'])

    # Select only the desired columns
    desired_columns = ['unique ID', 'Primary archt/firm'] + \
                      [col for col in result_df.columns if 'Primary archt/firm[' in col] + \
                      ['Primary archt/firm_flag']
    result_df = result_df[desired_columns]
    
    # Save the result to a CSV file
    result_df.to_csv(output_csv_path, index=False)
    print(f"All data saved to {output_csv_path}")

# Example usage
input_csv_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\Primary_archtfirm\\AAA_Proj_db_20231019-MSITM_Architech_input.csv'
output_csv_path = 'C:\\Users\\angel\\OneDrive\\Desktop\\Capstone\\Primary_archtfirm\\AAA_Proj_db_20231019-MSITM_Architech_curated_output.csv'
process_and_save_csv(input_csv_path, output_csv_path)


C:\Users\angel\AppData\Local\Temp\ipykernel_41236\1466083765.py:66: DtypeWarning: Columns (35,43,47,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv_path)


All data saved to C:\Users\angel\OneDrive\Desktop\Capstone\Primary_archtfirm\AAA_Proj_db_20231019-MSITM_Architech_curated_output.csv
